In [2]:
import sys
sys.path.insert(1, 'bin')

from persistence import sql
import functional as fn
import color as c

# Print all commands

In [ ]:
db = sql('functional')

for i in db.query('SELECT * FROM command'):
    print(c.blue + i['class'] + ':' + i['name'])
    print(c.white + fn.build(i['class'], i['name']))

db.close()

# Create command

if a command adds a new parameter it must be specified

In [ ]:
fn.create_command('python','if','''if({{condition}}):
  {{block}}''',['condition', 'block'])

# Create command and add child commands

(this command does not have valid sintax)

In [15]:
fn.create_command('debug', 'test')
fn.set_child(('debug', 'test'),('python','if'),{'block':('select','from')})

# Example

In [3]:
print(fn.build('python','if',{'condition':'true','block':'print("hello world")\nprint("line2")'}))

if(true):
  print("hello world")
  print("line2")


# Example2

In [3]:
obj = {
    'table'   : 'pessoa',
    'field'   : ['id','nome', 'sexo', 'idade'],
    'type'    : ['INT AUTO_INCREMENT', 'VARCHAR(32)', 'CHAR(1)', 'INT'],
    'value'   : ["@Id","@Name","@Sex","@Age"]
}

def cat(a, b, separator = ' '):
    return [a[i] + separator + b[i] for i in range(len(a))]

print(fn.build('sql','create', [obj['table'],cat(obj['field'],obj['type'])]) + ';\n')
print(fn.build('sql','insert values', [obj['table'],obj['field'][1::], obj['value'][1::]]) + ';\n')
print(fn.build('sql','select from', [obj['field'],obj['table']]) + ';\n')
print(fn.build('sql','update set where', [obj['table'], cat(obj['field'][1::], obj['value'][1::], ' = '), cat(obj['field'][:1], obj['value'][:1], ' = ')])+ ';\n')

CREATE TABLE 
  pessoa
(
  id INT AUTO_INCREMENT,
  nome VARCHAR(32),
  sexo CHAR(1),
  idade INT
);

INSERT INTO
  pessoa 
(
  nome,
  sexo,
  idade
)
VALUES
(
  @Name,
  @Sex,
  @AAge
);

SELECT
  id,
  nome,
  sexo,
  idade
FROM
  pessoa;

UPDATE
  pessoa
SET
 nome = @Name,
 sexo = @Sex,
 idade = @AAge
WHERE
  id = @Id;

